In [1]:
import json
import pandas
import os
import re
from collections import Counter
import matplotlib.pyplot as plt
import numpy as np
from itertools import tee, chain, islice
from math import log
from operator import itemgetter
from itertools import takewhile
import urllib
import requests
from tqdm import tqdm
import time

In [2]:
root_dir = '/home/marcin/Desktop/SemestrVIII/PJN'
year = "2018"
json_data_dir = f"{root_dir}/data/json"
filesInYearPattern = 'judgments-(316[3-9]|317\d)\.json'
out_dir = 'data/responses'

In [3]:
def judgement_texts(filename):
    with open(os.path.join(json_data_dir, filename), 'r') as jsonFile:
        judgements = json.load(jsonFile)['items']
    year_filtered = filter(lambda item: year in item['judgmentDate'],
                           judgements)
    yield from map(
        lambda item: (item['textContent'], item['judgmentDate'], item['id']),
        year_filtered)

In [4]:
def clean_text(line):
    notags = re.sub(r"<[^>]*>", " ", line)
    nobreaks = re.sub(r"-\n", " ", notags)
    noparens = re.sub("\(\.\.\.\)", " ", nobreaks)
    nonewline = re.sub("\\n", " ", noparens)
    return nonewline.replace(u'\xa0', u' ')

In [5]:
def send_request(text):
    url = "http://ws.clarin-pl.eu/nlprest2/base"
    lpmn = """any2txt|wcrft2|liner2({"model":"n82"})"""
    user = "mojadresemail"
    json_data = {
        'lpmn': lpmn,
        'user': user,
        'text': str(text.encode('utf-8'), 'utf-8')
    }
    resp = requests.post(
        url + '/process/',
        data=json.dumps(json_data),
        headers={'Content-Type': 'application/json'})
    return resp.text

In [9]:
def process_texts(texts):
    for _id, text in tqdm(texts):
        response = send_request(text)
        with open(os.path.join(out_dir, str(_id)), 'w') as out_file:
            out_file.write(response)
        time.sleep(60)

In [7]:
json_files = os.listdir(json_data_dir)
judgements_files = filter(lambda name: re.match(filesInYearPattern, name),
                          json_files)
text_date = list(chain.from_iterable(map(judgement_texts, judgements_files)))
to_process = sorted(text_date, key=itemgetter(1))[:100]
texts = [(t[2], clean_text(t[0])) for t in to_process]

In [10]:
process_texts(texts)


100%|██████████| 100/100 [2:38:51<00:00, 95.31s/it]
